# 2. EMOTION ANALYSIS
####  *Word-Based*

- **Assigns values for each emotion associated with each word in the reviews of the dataset**

- **Creates a csv file containing 11 columns, namely** 
    - **english word**
    - **anger** 
    - **fear**
    - **anticipation**
    - **trust**
    - **surprise**
    - **sadness**
    - **joy**
    - **disgust**
    - **sentence (number of sentence the word is part of)**
    - **review (number of review the word is part of)**

In [1]:
# -*- coding: utf-8 -*-

import csv
import json
import sys

In [2]:
# Files
LEXICON_FILE = 'NRC-Emotion-Lexicon-v0.92-English.csv'
CATEGORIES_FILE = 'categories.json'

TEXT_FILE = 'review_sentences.json'
DATA_FILE = 'output_data_sentences.csv'

----

In [3]:
# Init
vocabulary = []
words = []
categories = {}
category_headers = []
text = {}
data = []
chapters = []

In [4]:
# Read vocabulary
with open(LEXICON_FILE, 'r') as f:
    rows = csv.reader(f, delimiter=',')
    headers = next(rows, None) # remove header
    for row in rows:
        entry = {}
        for i, h in enumerate(headers):
            entry[h] = row[i]
        vocabulary.append(entry)
    #print (vocabulary)
    words = [v['\xef\xbb\xbfenglish word'] for v in vocabulary]

In [5]:
# Read categories
with open(CATEGORIES_FILE) as f:
    categories = json.load(f)
    emotions = categories['emotion']

In [6]:
# lemmatized filtered etc words from all reviews. list of lists
reviews = []
# Read normalized text
with open(TEXT_FILE) as f:
    text = json.load(f)
    for i in range(len(text)):
        reviews.append(text[i])

In [7]:
sent = 0
word = 0

for review in reviews:
    sent += len(review)
    for sentence in review:
        word += len(sentence)
    
print (sent, 'sentences', word, 'words')

(25246, 'sentences', 158838, 'words')


In [8]:
# Check if word matches any lexicons
def addData(word, review, sentence):
    global data
    global vocabulary
    global emotions
    global words

    match = -1
    for i,w in enumerate(words):
        if w==word:
            match = i
            break

    if match >= 0:
        for item in vocabulary:
            if item['\xef\xbb\xbfenglish word'] == word:
                entry = item
                break
            else: 
                continue
        row = [None]*11
        i = 0
        row[i] = entry['\xef\xbb\xbfenglish word']
        for emotion in emotions:
            i += 1
            if entry[emotion]:
                row[i] = entry[emotion]
            else:
                row[i] = -1
        row[i+1] = review
        row[i+2] = sentence
        data.append(row)

In [9]:
# Read each review
data = []
for i, review in enumerate(reviews):
    for j, sentence in enumerate(review):
        for word in sentence:
            addData(word, i, j)

In [10]:
import os
if os.path.isfile('output_data_sentences.csv'):
    os.remove('output_data_sentences.csv')

In [11]:
# Output data as csv
with open(DATA_FILE, 'w') as f:
    cw = csv.writer(f)
    headers = ['english word']
    for emotion in emotions:
        headers.append(emotion)
    headers.append(u'review')
    headers.append(u'sentence')
    cw.writerow(headers)
    for row in data:
        cw.writerow(row)
    print('Successfully wrote '+str(len(data))+' entries to file: '+DATA_FILE)

Successfully wrote 98213 entries to file: output_data_sentences.csv
